## Aperture photometry on the PAL5 globular cluster - epochs

This notebook now computes photometry on all available time epochs for the Palomar 5 globular cluster

In [1]:
import math
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits # FITS file management
from astropy.stats import sigma_clipped_stats # use within star detection
from photutils import aperture_photometry # used to perform photometry using apertures
from photutils import DAOStarFinder # star finding algorithm
from photutils import CircularAperture, CircularAnnulus

In [3]:
## copy from jake
base_dir = 'PAL5_data/*/'
files = []
for yep in glob.glob(base_dir + 'PAL5__e[0-9]_3p6um.fits', recursive=True) + glob.glob(base_dir + 'PAL5__e[0-9][0-9]_3p6um.fits', recursive=True):
    files.append(yep)
    print(yep)

print(files)

PAL5_data\PAL5__e1\PAL5__e1_3p6um.fits
PAL5_data\PAL5__e2\PAL5__e2_3p6um.fits
PAL5_data\PAL5__e3\PAL5__e3_3p6um.fits
PAL5_data\PAL5__e4\PAL5__e4_3p6um.fits
PAL5_data\PAL5__e5\PAL5__e5_3p6um.fits
PAL5_data\PAL5__e6\PAL5__e6_3p6um.fits
PAL5_data\PAL5__e7\PAL5__e7_3p6um.fits
PAL5_data\PAL5__e8\PAL5__e8_3p6um.fits
PAL5_data\PAL5__e9\PAL5__e9_3p6um.fits
PAL5_data\PAL5__e10\PAL5__e10_3p6um.fits
PAL5_data\PAL5__e11\PAL5__e11_3p6um.fits
PAL5_data\PAL5__e12\PAL5__e12_3p6um.fits
['PAL5_data\\PAL5__e1\\PAL5__e1_3p6um.fits', 'PAL5_data\\PAL5__e2\\PAL5__e2_3p6um.fits', 'PAL5_data\\PAL5__e3\\PAL5__e3_3p6um.fits', 'PAL5_data\\PAL5__e4\\PAL5__e4_3p6um.fits', 'PAL5_data\\PAL5__e5\\PAL5__e5_3p6um.fits', 'PAL5_data\\PAL5__e6\\PAL5__e6_3p6um.fits', 'PAL5_data\\PAL5__e7\\PAL5__e7_3p6um.fits', 'PAL5_data\\PAL5__e8\\PAL5__e8_3p6um.fits', 'PAL5_data\\PAL5__e9\\PAL5__e9_3p6um.fits', 'PAL5_data\\PAL5__e10\\PAL5__e10_3p6um.fits', 'PAL5_data\\PAL5__e11\\PAL5__e11_3p6um.fits', 'PAL5_data\\PAL5__e12\\PAL5__e12_3p6u

In [4]:
## BASE DATA DIRECTORY ##

base_dir = 'PAL5_data/*/'
channel = '3p5um'

## PARAMETERS ##
sigma_val = 6.
fwhm = 5.
r_ap = 6.
r_in = 6.
r_out = 14.
roundlo = -0.5
roundhi = 0.5

## LOOP COUNTER ##
#counter = []
i = 0

## FITS FILE SELECTION ##
for file in glob.glob(base_dir+'PAL5__e[0-9]_'+channel+'.fits', recursive = True) + glob.glob(base_dir+'PAL5__e[0-9][0-9]_'+channel+'.fits', recursive = True):
    i += 1
    print('EPOCH NUMBER: {}'.format(i))
    ## OPENING FITS FILE AND EXTRACTING DATA ##
    with fits.open(file) as header_list:
        header = header_list[0].header
        fluxconv = header['FLUXCONV']
        exptime = header['EXPTIME']
        counts = exptime / fluxconv
        image_data = fits.getdata(file, ext = 0)
        data = image_data * counts
        print('FLUXCONV = {}\nEXPTIME = {}'.format(fluxconv, exptime))

    ## SOURCE DETECTION ##
    mean_val, median_val, std_val = sigma_clipped_stats(data, sigma = sigma_val)
    
    daofind = DAOStarFinder(fwhm = fwhm, threshold = sigma_val * std_val, roundlo = roundlo, roundhi = roundhi)
    sources = daofind(data) #- median_val) # necessary here?
    print('Number of stars detected: {}'.format(len(sources)))
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r = 6.)

    plt.imshow(data, cmap = 'viridis', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')
    plt.colorbar(fraction = 0.05)
    apertures.plot(color = 'black', lw = 1., alpha = .75)
    plt.title('Aperture photometry on epoch {} in channel {}: sigma = {}, fwhm = {}, roundlo = {}, roundhi = {}'
              .format(i, channel, sigma_val, fwhm, roundlo, roundhi))
    plt.grid(b = True, which = 'major', lw = .5, alpha = .4, color = 'black')
    plt.gcf().set_size_inches(15, 8)
    plt.savefig(r'aperture_photometry_output_images/aperture_phot_params_img{}.png'.format(i))
    plt.show()
    plt.close()
    
    ## APERTURE PHOTOMETRY ##
    
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    circular_apertures = CircularAperture(positions, r = 6.)
    annuli_apertures = CircularAnnulus(positions, r_in = 6., r_out = 14.)
    apertures = [circular_apertures, annuli_apertures]

    # initial aperture photometry table
    phot_init = aperture_photometry(data, apertures)

    # background subtraction using sigma-clipped median and annuli
    annulus_masks = annuli_apertures.to_mask(method = 'center')

    bkg_median = []
    for mask in annulus_masks:
        annulus_data = mask.multiply(data)
        annulus_data_1d = annulus_data[mask.data > 0] # extract 1D array of data values
        _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d) # utilise sigma clipping on the annulus masks
        bkg_median.append(median_sigclip)

    bkg_median = np.array(bkg_median)
    # now append bkg_median, aperture background and aperture sum background values to photometry data
    phot_init['annulus_median'] = bkg_median
    phot_init['aper_bkg'] = bkg_median * circular_apertures.area
    phot_init['aper_sum_bkgsub'] = phot_init['aperture_sum_0'] - phot_init['aper_bkg']

    ## APPARENT MAGNITUDE ##
    
    aper_corr = 1.125 # aperture correction for 337 (6,6,14) apertures in channel 1, given in IRAC handbook §4.10
    zmag = 18.8       # from IRAC handbook §4.8

    phot = phot_init                    # redefine photometry table for ease
    phot['bkgsub_flux'] = float('NaN')  # populate new column to convert into flux
    phot['apparent_mag'] = float('NaN') # populate a new table (very quirky here, nans?)

    for i in range(0, len(phot)):
        phot['bkgsub_flux'][i] = phot['aper_sum_bkgsub'][i] * fluxconv / exptime
        for i in range(0, len(phot)):
            if phot['bkgsub_flux'][i] >= 0:
                phot['apparent_mag'][i] = zmag - 2.5 * math.log10(phot['bkgsub_flux'][i] * aper_corr)

    # export into csv file
    phot['id', 'xcenter', 'ycenter', 'apparent_mag'].write(r'C:\Users\lukeb\Documents\MPhys_Project\output_files\aperphot_epoch{}_{}.txt'.format(i, channel), format = 'csv', overwrite = True)
    print(phot)

In [8]:
## BASE DATA DIRECTORY ##

base_dir = 'PAL5_data/*/'
channel = '3p5um'

## PARAMETERS ##
sigma_val = 6.
fwhm = 5.
r_ap = 6.
r_in = 6.
r_out = 14.
roundlo = -0.5
roundhi = 0.5

## LOOP COUNTER ##
#counter = []
i = 0


## FITS FILE SELECTION ##
for file in glob.glob(base_dir+'PAL5__e[0-9]_'+channel+'.fits', recursive = True) + glob.glob(base_dir+'PAL5__e[0-9][0-9]_'+channel+'.fits', recursive = True):
    file = file
    i += 1
    print('EPOCH NUMBER: {}'.format(i))
    print(file)